In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/dataset

/content/drive/MyDrive/dataset


In [3]:
!pip install tensorflow
!pip install opencv-python

In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define paths to the dataset
accidented_path = 'accidented'
nonaccident_path = 'nonaccident'

# Function to load and preprocess images
def load_images_from_folder(folder, label, target_size=(128, 128)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, target_size)
            img = img / 255.0  # Normalize to [0, 1]
            images.append(img)
            labels.append(label)
    return images, labels

# Load images and labels
accidented_images, accidented_labels = load_images_from_folder(accidented_path, 1)
nonaccident_images, nonaccident_labels = load_images_from_folder(nonaccident_path, 0)

# Combine the data
images = np.array(accidented_images + nonaccident_images)
labels = np.array(accidented_labels + nonaccident_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [5]:
# Build the Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Train the Model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100, validation_data=(X_test, y_test))


Epoch 1/100
10/10 [==============================] - 14s 1s/step - loss: 0.6967 - accuracy: 0.6700 - val_loss: 0.5509 - val_accuracy: 0.7632
Epoch 2/100
10/10 [==============================] - 12s 1s/step - loss: 0.6136 - accuracy: 0.7100 - val_loss: 0.5613 - val_accuracy: 0.7632
Epoch 3/100
10/10 [==============================] - 13s 1s/step - loss: 0.6172 - accuracy: 0.7100 - val_loss: 0.5942 - val_accuracy: 0.7632
Epoch 4/100
10/10 [==============================] - 12s 1s/step - loss: 0.6069 - accuracy: 0.7100 - val_loss: 0.5453 - val_accuracy: 0.7632
Epoch 5/100
10/10 [==============================] - 11s 1s/step - loss: 0.6041 - accuracy: 0.7100 - val_loss: 0.5334 - val_accuracy: 0.7632
Epoch 6/100
10/10 [==============================] - 11s 1s/step - loss: 0.5898 - accuracy: 0.7100 - val_loss: 0.5239 - val_accuracy: 0.7632
Epoch 7/100
10/10 [==============================] - 12s 1s/step - loss: 0.5646 - accuracy: 0.7300 - val_loss: 0.4901 - val_accuracy: 0.7632
Epoch 8/100
1

In [7]:
# Save the Model
model.save('car_accident_detection_model.h5')